In [ ]:
# from functools import lru_cache
# import sys
# sys.setrecursionlimit(10**9)
# input = sys.stdin.readline
from decimal import Decimal
from functools import cmp_to_key
from collections import Counter
from itertools import permutations
from itertools import combinations
import math
import heapq
from collections import OrderedDict
from collections import deque
import bisect

from collections import defaultdict

# op: op(x,y) return x*y みたいな演算
# e: 単位元，x+e = e+x = x になるようなe
# n: 要素数
# v: 要素の配列
# セグメント木
class SegTree:
    def __init__(self, op, e, n, v=None):
        self._n = n
        self._op = op
        self._e = e
        self._log = (n - 1).bit_length()
        self._size = 1 << self._log
        self._d = [self._e()] * (self._size << 1)
        if v is not None:
            for i in range(self._n):
                self._d[self._size + i] = v[i]
            for i in range(self._size - 1, 0, -1):
                self._d[i] = self._op(self._d[i << 1], self._d[i << 1 | 1])
    # p番目の要素をxに変更
    def set(self, p, x):
        p += self._size
        self._d[p] = x
        while p:
            self._d[p >> 1] = self._op(self._d[p], self._d[p ^ 1])
            p >>= 1
    # p番目の要素を返す
    def get(self, p):
        return self._d[p + self._size]
    # op(d[l], d[l+1], ..., d[r-1])を返す
    def prod(self, l, r):
        sml, smr = self._e(), self._e()
        l += self._size
        r += self._size
        while l < r:
            if l & 1:
                sml = self._op(sml, self._d[l])
                l += 1
            if r & 1:
                r -= 1
                smr = self._op(self._d[r], smr)
            l >>= 1
            r >>= 1
        return self._op(sml, smr)
    # op(d[0], d[1], ..., d[n-1])を返す
    def all_prod(self):
        return self._d[1]

def main():
    N = int(input())
    fir_list = [tuple()] * N
    fir_dict = defaultdict(int)
    sec_dict = defaultdict(int)
    thr_dict = defaultdict(int)
    for i in range(N):
        fir_sorted = sorted(map(int, input().split()))
        fir_list[i] = tuple(fir_sorted)
        fir_dict[fir_sorted[0]] += 1
        sec_dict[fir_sorted[1]] += 1
        thr_dict[fir_sorted[2]] += 1
    fir_list.sort()
    # 高さ，幅，奥行きをそれぞれ座標圧縮
    comp0_dict = defaultdict(int)
    comp1_dict = defaultdict(int)
    comp2_dict = defaultdict(int)
    for i,key in enumerate(sorted(fir_dict.keys())):
        comp0_dict[key] = i+1
    for i,key in enumerate(sorted(sec_dict.keys())):
        comp1_dict[key] = i+1
    for i,key in enumerate(sorted(thr_dict.keys())):
        comp2_dict[key] = i+1
    for i, fir in enumerate(fir_list):
        fir_list[i] = (comp0_dict[fir[0]], comp1_dict[fir[1]], comp2_dict[fir[2]])
    # 小さいへんから見ていく，segtreeで2番目の要素と3番目の要素の最小値を管理
    min_tree = SegTree(min, lambda: 10**6, 10**5)
    for fir in fir_list:
        # min_treeの中で，2番目の要素がfir[1]未満のものの中で，3番目の要素が最小のものを探す
        min_fir2 = min_tree.prod(0, fir[1])
        # その値がfir[2]未満ならOK
        if min_fir2 < fir[2]:
            print("Yes")
            return
        # そうでないなら，fir[2]がmin_treeのfir[1]番目より小さかったら更新
        else:
            if min_tree.get(fir[1]) > fir[2]:
                min_tree.set(fir[1], fir[2])
    print("No")
    
if __name__ == "__main__":
    main()